<a href="https://colab.research.google.com/github/ahmetcanerdem/Bil470Proje/blob/master/Image%20Generator%20TRANSFER-CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
tf.compat.v1.disable_eager_execution()

#**SETTING UP THE DATA GENERATORS TO GET THE DATA**

In [39]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from google.colab import drive

##iv3 ün weight değerleri
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
from tensorflow.keras.applications.inception_v3 import InceptionV3

weight_file= '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'



drive.mount('/content/drive')

input_size=(128,128)
test_path=''
test_path='./drive/My Drive/test_resampled/hair_remove/'
training_path='./drive/My Drive/train_resampled/hair_remove/'
train_0_p=training_path+'train_0/'
train_1_p=training_path+'train_1_large/'
test_0_path= test_path + 'train_0_test/'
test_1_path= test_path + 'train_1_test_large/'
treshold=0.55
image_dim=3
###############DATA GENERATOR#############################

from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_generator = ImageDataGenerator(rescale=1./255)
train_generator = image_generator.flow_from_directory(
    training_path,
    target_size=(128,128),
    batch_size = 256,
    class_mode = 'binary'
)

validation_generator = image_generator.flow_from_directory(
    test_path,
    target_size=(128,128),
    batch_size = 256,
    class_mode = 'binary'
)






X_steps = int(train_generator.samples/512)+1
Y_steps = int(validation_generator.samples/512)+1



X_steps
Y_steps



#########################################################################################################



--2020-07-24 14:39:49--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.112.128, 74.125.124.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.112.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   113MB/s    in 0.7s    

2020-07-24 14:39:50 (113 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 11954 images belonging to 2 classes.
Found 1005 images belonging to 2 classes.


2

# **TRAINING CNN ONLY Learning Model Over Big Data**

> Data has resampled and dehaired.

> In each epoch model fits itself to 5000 samples of the data




In [43]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop



model =tf.keras.Sequential([
                     tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(input_size[0],input_size[1],image_dim)),
                     tf.keras.layers.MaxPooling2D(2,2),
                     tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                     tf.keras.layers.MaxPooling2D(2,2),
                     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                     tf.keras.layers.MaxPooling2D(2,2),
                     tf.keras.layers.Flatten(),
                     tf.keras.layers.Dense(512, activation='relu'),
                     tf.keras.layers.Dense(1024, activation='relu'),
                     tf.keras.layers.Dense(1, activation='sigmoid'),

])

In [29]:
history =model.fit(
    train_generator,
    steps_per_epoch= 8,
    epochs=20,
    validation_data = validation_generator,
    validation_steps= Y_steps,
)

Epoch 1/20
8/8 [==============================] - 433s 54s/step - loss: 0.7223 - acc: 0.5435 - val_loss: 0.7960 - val_acc: 0.1738
Epoch 2/20
8/8 [==============================] - 89s 11s/step - loss: 0.6605 - acc: 0.5990 - val_loss: 0.5465 - val_acc: 0.7480
Epoch 3/20
8/8 [==============================] - 171s 21s/step - loss: 0.6330 - acc: 0.6440 - val_loss: 0.6541 - val_acc: 0.6328
Epoch 4/20
8/8 [==============================] - 172s 22s/step - loss: 0.6150 - acc: 0.6665 - val_loss: 0.4414 - val_acc: 0.8105
Epoch 5/20
8/8 [==============================] - 194s 24s/step - loss: 0.6142 - acc: 0.6631 - val_loss: 0.5058 - val_acc: 0.7324
Epoch 6/20
8/8 [==============================] - 170s 21s/step - loss: 0.6010 - acc: 0.6597 - val_loss: 0.7968 - val_acc: 0.4844
Epoch 7/20
8/8 [==============================] - 29s 4s/step - loss: 0.5818 - acc: 0.7017 - val_loss: 0.5525 - val_acc: 0.6895
Epoch 8/20
8/8 [==============================] - 29s 4s/step - loss: 0.6041 - acc: 0.6592 - 

*Reached max 81% and average 75% acuracy in the validation data*

# **TRAINING Transfer Learning Model Over Big Data**

> Data has resampled and dehaired.

> In each epoch model fits itself to 5000 samples of the data




In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
model_I3_trained= InceptionV3(input_shape= (input_size[0], input_size[1],image_dim),
                      include_top= False,
                      weights= None
                      )

model_I3_trained.load_weights(weight_file)
for layer in model_I3_trained.layers:
  layer.trainable= False


#model_I3_trained.summary()



from tensorflow.keras.optimizers import RMSprop

last_layer= model_I3_trained.get_layer('mixed7')
last_output= last_layer.output

x= layers.Flatten()(last_output)
x= layers.Dense(2048,activation='relu')(x)
x= layers.Dense(1, activation='sigmoid')(x)


model_T= Model(model_I3_trained.input, x)
model_T.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['acc'])

model_T.summary()

In [42]:
history_T=model_T.fit(
    train_generator,
    steps_per_epoch= 16,
    epochs=20,
    validation_data = validation_generator,
    validation_steps= Y_steps,
)

Epoch 1/20
16/16 [==============================] - 134s 8s/step - loss: 0.4264 - acc: 0.8423 - val_loss: 0.7353 - val_acc: 0.8301
Epoch 2/20
16/16 [==============================] - 131s 8s/step - loss: 0.1499 - acc: 0.9592 - val_loss: 1.1143 - val_acc: 0.8223
Epoch 3/20
16/16 [==============================] - 134s 8s/step - loss: 0.1150 - acc: 0.9692 - val_loss: 0.9566 - val_acc: 0.8262
Epoch 4/20
16/16 [==============================] - 132s 8s/step - loss: 0.0589 - acc: 0.9823 - val_loss: 2.1092 - val_acc: 0.8223
Epoch 5/20
16/16 [==============================] - 134s 8s/step - loss: 0.0866 - acc: 0.9780 - val_loss: 1.0172 - val_acc: 0.8281
Epoch 6/20
16/16 [==============================] - 133s 8s/step - loss: 0.0553 - acc: 0.9829 - val_loss: 1.5483 - val_acc: 0.8262
Epoch 7/20
16/16 [==============================] - 134s 8s/step - loss: 0.0351 - acc: 0.9912 - val_loss: 1.3730 - val_acc: 0.8301
Epoch 8/20
16/16 [==============================] - 133s 8s/step - loss: 0.0695 - a

*Reached max 83% accuracy in the validation set and 100% in training set seems pretty good*


In [37]:
def findAcc(m_model, data, dataclasses, treshold =0.55):
  pM= m_model.predict(data)
  cM=[]
  for i in pM:
    if i > treshold:
      cM.append(1)
    else:
      cM.append(0)
  
  c=0
  for i in range(cM):
    if (cM[i]==dataclasses[i]):
      c=c+1;
  
  return (c/len(dataclasses))





'binary_crossentropy'